In [1]:
import os, sys, pandas as pd
sys.path += ['../aux_scripts']

from models_to_run import *

In [2]:
model_typology = pd.read_csv('../model_opts/model_typology.csv')

In [3]:
model_contrasts = model_typology.copy()#[['architecture','model','train_type','train_data']]
model_contrasts['model_string'] = model_contrasts['model'] + '_' + model_contrasts['train_type']

def add_random(model_string):
    return model_string.replace('classification','random')

model_contrasts = model_contrasts[model_contrasts['model_string'].isin(models_to_run_redux) | 
                                  model_contrasts['model_string'].isin(map(add_random, models_to_run_redux)) |
                                  model_contrasts['model_string'].str.contains('ipcl')]

random_list = model_contrasts[model_contrasts['train_type'].isin(['random'])]
def compare_training(row):
    if row['model'] in random_list.model.to_list():
        return row['train_type']
    else:
        return None

def compare_architecture(row):
    if row['train_data'] != 'imagenet' or row['train_type'] != 'classification':
        return None
    else:
        return row['model_class']
    
def compare_goal_slip(row):
    if row['model_source'] != 'slip':
        return None
    elif 'CC12M' in row['model']:
        return None
    else:
        return row['model'].split('-')[2]
    
def compare_goal_taskonomy_tasks(row):
    if row['model_source'] != 'taskonomy':
        return None
    else:
        return row['model']
    
def compare_goal_taskonomy_cluster(row):
    if row['model_source'] != 'taskonomy':
        return None
    else:
        return row['task_cluster']

def compare_goal_expertise(row):
    if row['model_source'] != 'bit_expert':
        return None
    else:
        return row['model'].split('-')[-1]
    
def compare_diet_ipcl(row):
    if row['model_source'] != 'open_ipcl':
        return None
    else:
        return row['train_data']
    
def compare_goal_selfsupervised(row):
    if (row['model_source'] == 'vissl' and row['architecture'] == 'ResNet50' 
        and row['train_data'] == 'imagenet'):
        return '-'.join(row['model'].split('-')[1:])
    else:
        return None

contrastive_list = ['PIRL','NPID-4KNegative','SimCLR','MoCoV2-BS256','NPID-32KNegative',
                   'SwAV-BS4096-2x224+6x96','BarlowTwins-BS2048','DeepClusterV2-2x224+6x96']

def compare_goal_contrastive(row):
    if (row['model_source'] == 'vissl' and row['architecture'] == 'ResNet50' 
        and row['train_data'] == 'imagenet'):
        model = '-'.join(row['model'].split('-')[1:])
        return 'Contrastive' if model in contrastive_list else 'Non-Contrastive'
    else:
        return None

imagenet21k_list = model_contrasts[model_contrasts['train_data'] == 'imagenet21k']['architecture'].to_list()
imagenet21k_list.pop(imagenet21k_list.index('vit_base_r50_s16_224'))
def compare_diet_imagenetsize(row):
    if row['architecture'] in imagenet21k_list:
        return row['train_data']
    else:
        return None
    
compare_funcs = [key for key,value in locals().items() if 'compare' in key and callable(value)]

for compare_func in compare_funcs:
    row_func = locals()[compare_func]
    model_contrasts[compare_func] = model_contrasts.apply(lambda row: row_func(row), axis = 1)

In [4]:
model_contrasts.compare_training.value_counts()

classification    65
random            65
Name: compare_training, dtype: int64

In [5]:
model_contrasts.compare_architecture.value_counts()

Convolutional    34
Transformer      22
MLP-Mixer         6
Hybrid            3
Name: compare_architecture, dtype: int64

In [6]:
model_contrasts.compare_goal_slip.value_counts()

SimCLR    3
CLIP      3
SLIP      3
Name: compare_goal_slip, dtype: int64

In [7]:
model_contrasts.compare_goal_taskonomy_tasks.value_counts()

autoencoding        1
keypoints2d         1
vanishing_point     1
segment_unsup2d     1
segment_unsup25d    1
segment_semantic    1
room_layout         1
reshading           1
point_matching      1
normal              1
nonfixated_pose     1
keypoints3d         1
jigsaw              1
class_object        1
inpainting          1
fixated_pose        1
egomotion           1
edge_texture        1
edge_occlusion      1
depth_zbuffer       1
depth_euclidean     1
denoising           1
curvature           1
class_scene         1
random_weights      1
Name: compare_goal_taskonomy_tasks, dtype: int64

In [8]:
model_contrasts.compare_goal_taskonomy_cluster.value_counts()

3D           8
Geometric    7
2D           5
Semantic     3
Other        1
Random       1
Name: compare_goal_taskonomy_cluster, dtype: int64

In [9]:
model_contrasts.compare_goal_expertise.value_counts()

Food           1
Vehicle        1
Instrument     1
Flower         1
Animal         1
Object         1
Bird           1
Mammal         1
Arthropod      1
Relation       1
Abstraction    1
Name: compare_goal_expertise, dtype: int64

In [10]:
model_contrasts.compare_goal_selfsupervised.value_counts()

JigSaw-P100                 1
JigSaw-Goyal19              1
RotNet                      1
ClusterFit-16K-RotNet       1
PIRL                        1
SimCLR                      1
DeepClusterV2-2x224+6x96    1
SwAV-BS4096-2x224+6x96      1
MoCoV2-BS256                1
BarlowTwins-BS2048          1
Name: compare_goal_selfsupervised, dtype: int64

In [11]:
model_contrasts.compare_goal_contrastive.value_counts()

Contrastive        6
Non-Contrastive    4
Name: compare_goal_contrastive, dtype: int64

In [12]:
model_contrasts.compare_diet_ipcl.value_counts()

imagenet      1
openimages    1
places256     1
vggface2      1
Name: compare_diet_ipcl, dtype: int64

In [13]:
model_contrasts.compare_diet_imagenetsize.value_counts()

imagenet       12
imagenet21k    12
Name: compare_diet_imagenetsize, dtype: int64

In [14]:
model_contrasts[model_contrasts.architecture.str.contains('50')];

In [15]:
model_contrasts.to_csv('../model_contrasts.csv', index = None)